In [1]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [2]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [3]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/primary categories - Train.npz')
data_validation = np.load(r'Full Dataset/primary categories - Validation.npz')
data_test = np.load(r'Full Dataset/primary categories - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']


images_test = data_test['images']
labels_test = data_test['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [4]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 1

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete(['32','64']))

METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [5]:
#CREATE THE MODEL AND TRAIN IT

#Stop the model from overfitting ie whenever the validation loss increases
#the code tells the model to stop when the val_loss starts to increase in two subsequent epochs

#directory to store the cm logs  




session_num = 1
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(32, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(3)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/Model 1/fit/run-{session_num}"

    
  


 #Create a Confusion Matrix
        
        
        
#Plot_to_image

    def plot_to_cm_image(cm, class_labels):
        
        fig, ax = plt.subplots(figsize=(6, 6))
       

        sns.set(font_scale=1.2)  # Adjust the font size
        sns.heatmap(cm, annot=True, fmt="0.2f", cmap="Blues", square=True,
                    xticklabels=class_labels,
                    yticklabels=class_labels, ax=ax)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.close(fig)
               
        
# Convert the plot to a NumPy array
    
        fig.canvas.draw()
        plt_image = np.array(fig.canvas.renderer.buffer_rgba())
#         plt_image = np.frombuffer(plt.gcf().canvas.tostring_rgb(), dtype=np.uint8)
#         plt_image = plt_image.reshape(plt.gcf().canvas.get_width_height()[::-1] + (3,))
        
        #Convert the array image to a tensor image of 4 dimensions for compartibility with TensorFlow
        image_tensor = tf.convert_to_tensor(plt_image)  # Convert to a TensorFlow tensor
        image_tensor = tf.expand_dims(image_tensor, axis=0)  # Add the batch dimension

        return image_tensor
    
       
        

        
        
    # log the confusion matrix as a heatmap
    
    confusion_matrix_writer = tf.summary.create_file_writer(log_dir +'/cm')

    def log_confusion_matrix_to_tensorboard(epoch,log=None):
        val_predict_raw= model.predict(images_val)
        val_predict = np.argmax(val_predict_raw, axis =1)


        
        
        #Get the confusion matrix
        cm_raw = confusion_matrix(labels_val, val_predict).astype('float')
        #Normalize the confusion matrix(cm) row-wise
        cm = cm_raw/cm_raw.sum(axis=1)[:,np.newaxis]
        cm = np.round(cm, 2)
        
        
        
        class_labels= ["Glasses/Sunglassses", "Trousers/Jeans", "Shoes"]
        cm_image = plot_to_cm_image(cm, class_labels)

        #Log the cm as an iomage summary
        with confusion_matrix_writer.as_default():

            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            


    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix_to_tensorboard)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    model.save(f"Saved/Model 1/run-{session_num}")


    return accuracy



In [6]:
#Log the hyperparameter to the TensorBoard


def run(log_dir,hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num:1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': '32'}
51/51 [==============================] - 2s 29ms/step
203/203 - 41s - loss: 0.0864 - accuracy: 0.9722 - val_loss: 0.0062 - val_accuracy: 0.9994 - 41s/epoch - 201ms/step
51/51 [==============================] - 2s 30ms/step - loss: 0.0062 - accuracy: 0.9994
INFO:tensorflow:Assets written to: Saved/Model 1/run-1\assets


INFO:tensorflow:Assets written to: Saved/Model 1/run-1\assets


---Starting trial: run-2
{'filter_size': 3, 'filter_num': '64'}
51/51 [==============================] - 2s 29ms/step
203/203 - 41s - loss: 0.0967 - accuracy: 0.9674 - val_loss: 0.0048 - val_accuracy: 0.9988 - 41s/epoch - 203ms/step
51/51 [==============================] - 2s 32ms/step - loss: 0.0048 - accuracy: 0.9988
INFO:tensorflow:Assets written to: Saved/Model 1/run-2\assets


INFO:tensorflow:Assets written to: Saved/Model 1/run-2\assets


In [7]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model 1/hparam_tuning'


In [8]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model 1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
